In [19]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

In [20]:
data_path=None
all_variables=None
test_path=None
# target='search_term'
target=None
nthreads=1 
min_mem_size=6 
run_time=500
classification=True
scale=True
max_models=20    
model_path=None
balance_y=False 
balance_threshold=0.2
name=None 
server_path=None  
analysis=0

In [21]:
#Funtion for meta data
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,
                  balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data




In [22]:
#Function to convert dictionary into json file
def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
    
#Function for random alphabet   
def alphabet(n):
    alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
    str=''
    r=len(alpha)-1   
    while len(str)<n:
        i=random.randint(0,r)
        str+=alpha[i]   
        return str

def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


    

def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val
    return d    

In [23]:
def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id  
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    

In [24]:
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm


def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    

In [25]:
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()     
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d

In [26]:

    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d

In [27]:
all_variables=None

In [28]:
data_path='C:/Users/pkash/Downloads/adult.csv'

In [29]:
run_id=alphabet(9)
if server_path==None:
    server_path=os.path.abspath(os.curdir)
os.chdir(server_path) 
run_dir = os.path.join(server_path,run_id)
os.mkdir(run_dir)
os.chdir(run_dir)    

# run_id to std out
print (run_id)

7


In [30]:
##For logs
logfile=run_id+'_autoh2o_log.zip'
logs_path=os.path.join(run_dir,'logs')
print(logs_path,' ',logfile)

C:\Users\pkash\7\logs   7_autoh2o_log.zip


In [31]:
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.201-b09, mixed mode)
  Starting server from C:\Users\pkash\Anaconda\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\pkash\AppData\Local\Temp\tmprsk_98ok
  JVM stdout: C:\Users\pkash\AppData\Local\Temp\tmprsk_98ok\h2o_pkash_started_from_python.out
  JVM stderr: C:\Users\pkash\AppData\Local\Temp\tmprsk_98ok\h2o_pkash_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_pkash_iv8y8i
H2O cluster total nodes:,1
H2O cluster free memory:,1.743 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [32]:
# meta data
meta_data = set_meta_data(analysis, run_id,server_path,data_path,test_path,model_path,target,run_time,classification,scale,max_models,balance_y,balance_threshold,name,run_dir,nthreads,min_mem_size)
print(meta_data)

{'start_time': 1555261986.631968, 'target': None, 'server_path': 'C:\\Users\\pkash', 'data_path': 'C:/Users/pkash/Downloads/adult.csv', 'test_path': None, 'max_models': 9, 'run_time': 500, 'run_id': '7', 'scale': False, 'classification': False, 'model_path': None, 'balance': False, 'balance_threshold': 0.2, 'project': None, 'end_time': 1555261986.631968, 'execution_time': 0.0, 'run_path': 'C:\\Users\\pkash\\7', 'nthreads': 1, 'min_mem_size': 6, 'analysis': 0}


In [33]:
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [34]:
df.head()

age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K
63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,>50K
24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,<=50K


In [35]:

df.describe()

Rows:48842
Cols:15




,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
type,int,enum,int,enum,int,enum,enum,enum,enum,enum,int,int,int,enum,enum
mins,17.0,,12285.0,,1.0,,,,,,0.0,0.0,1.0,,
mean,38.643585438761704,,189664.13459727284,,10.078088530363212,,,,,,1079.0676262233324,87.50231358257237,40.42238237582409,,
maxs,90.0,,1490400.0,,16.0,,,,,,99999.0,4356.0,99.0,,
sigma,13.710509934443555,,105604.02542315728,,2.570972755592256,,,,,,7452.019057655393,403.00455212435907,12.391444024252307,,
zeros,0,,0,,0,,,,,,44807,46560,0,,
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,25.0,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38.0,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28.0,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K


In [36]:
# dependent variable
# assign target and inputs for classification or regression
if target==None:
    target=df.columns[14]   
y = target
y

'income'

In [37]:
x = list(df.columns)
x.remove(y)
x
# Add independent variables

meta_data['x']=x 
# impute missing values

_=impute_missing_values(df,x, scale)


In [39]:
# # impute missing values

# _ = df[ints].impute(method='median')

# if scale:
    
#     df[ints] = df[ints].scale()

In [40]:
if analysis == 3:
  classification=False
elif analysis == 2:
  classification=True
elif analysis == 1:
  classification=True

In [41]:
def check_y(y,df):
  ok=False
  C = [name for name in df.columns if name == y]
  for key, val in df.types.items():
    if key in C:
      if val in ['real','int','enum']:        
        ok=True         
  return ok

In [42]:
ok=check_y(y,df)
if not ok:
    print(ok)

In [43]:
classification=True
if classification:
    print(df[y].levels())

[['<=50K', '>50K']]


In [44]:
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    

In [45]:
allV=get_variables_types(df)
allV

{'age': 'int',
 'workclass': 'enum',
 'fnlwgt': 'int',
 'education': 'enum',
 'educational-num': 'int',
 'marital-status': 'enum',
 'occupation': 'enum',
 'relationship': 'enum',
 'race': 'enum',
 'gender': 'enum',
 'capital-gain': 'int',
 'capital-loss': 'int',
 'hours-per-week': 'int',
 'native-country': 'enum',
 'income': 'enum'}

In [46]:
meta_data['variables']=allV

In [47]:
# Set up AutoML

aml = H2OAutoML(max_runtime_secs=run_time,project_name = name)

In [48]:
model_start_time = time.time()

In [49]:
aml.train(x=x,y=y,training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [51]:
meta_data['model_execution_time'] = time.time() - model_start_time

In [52]:
# get leaderboard
aml_leaderboard_df=aml.leaderboard.as_data_frame()

In [53]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190414_131354,0.928743,0.277279,0.167143,0.296535,0.087933
1,GBM_1_AutoML_20190414_131354,0.928644,0.277369,0.160988,0.296409,0.087858
2,GBM_3_AutoML_20190414_131354,0.928374,0.277844,0.165077,0.296980,0.088197
3,StackedEnsemble_AllModels_AutoML_20190414_131354,0.927854,0.292921,0.177949,0.300626,0.090376
4,GBM_grid_1_AutoML_20190414_131354_model_6,0.927796,0.279340,0.172041,0.297485,0.088498
5,GBM_grid_1_AutoML_20190414_131354_model_4,0.927535,0.279859,0.173029,0.297787,0.088677
6,StackedEnsemble_BestOfFamily_AutoML_20190414_1...,0.927298,0.294950,0.180466,0.301158,0.090696
7,GBM_4_AutoML_20190414_131354,0.927245,0.279931,0.169880,0.297962,0.088782
8,GBM_5_AutoML_20190414_131354,0.926568,0.280946,0.173531,0.298737,0.089244
9,GBM_grid_1_AutoML_20190414_131354_model_7,0.926406,0.281980,0.169133,0.298683,0.089212


In [54]:
# STart best model as first model

model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [55]:
mod_best._id

'GBM_2_AutoML_20190414_131354'

In [56]:
# Get stacked ensemble  
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
se=get_stacked_ensemble(model_set)

In [57]:
print(se)

StackedEnsemble_BestOfFamily_AutoML_20190414_131354


In [58]:
if se is not None:
  mod_best=h2o.get_model(se)

In [59]:

dir(mod_best)

['F0point5',
 'F1',
 'F2',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bc',
 '_bcin',
 '_check_and_save_parm',
 '_check_targets',
 '_compute_algo',
 '_end_time',
 '_estimator_type',
 '_future',
 '_get_metrics',
 '_have_mojo',
 '_have_pojo',
 '_id',
 '_is_xvalidated',
 '_job',
 '_keyify_if_h2oframe',
 '_metrics_class',
 '_model_json',
 '_parms',
 '_plot',
 '_requires_training_frame',
 '_resolve_model',
 '_run_time',
 '_start_time',
 '_train',
 '_verify_training_frame_params',
 '_xval_keys',
 'accuracy',
 'actual_params',
 'aic',
 'algo',
 'auc',
 'base_models',
 'biases',
 'blending_frame',
 'catoffsets',
 'coef',
 'coef_norm',
 'confusion_matrix',

In [60]:
mod_best._id

'StackedEnsemble_BestOfFamily_AutoML_20190414_131354'

In [61]:
mod_best._get_metrics

<function h2o.model.model_base.ModelBase._get_metrics(o, train, valid, xval)>

In [62]:
type(mod_best)

h2o.estimators.stackedensemble.H2OStackedEnsembleEstimator

In [63]:
mods=mod_best.coef_norm
print(mods)

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190414_131354
No model summary for this model


ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.07604957174062595
RMSE: 0.2757708681870258
LogLoss: 0.2513496130824789
Null degrees of freedom: 48841
Residual degrees of freedom: 48838
Null deviance: 53750.68147229072
Residual deviance: 24552.835604348867
AIC: 24560.835604348867
AUC: 0.9515424855534957
pr_auc: 0.8268156738955146
Gini: 0.9030849711069915
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3232944201778553: 


,<=50K,>50K,Error,Rate
<=50K,34159.0,2996.0,0.0806,(2996.0/37155.0)
>50K,2447.0,9240.0,0.2094,(2447.0/11687.0)
Total,36606.0,12236.0,0.1114,(5443.0/48842.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3232944,0.7724784,232.0
max f2,0.1221157,0.8438620,322.0
max f0point5,0.6759068,0.8072431,117.0
max accuracy,0.4524263,0.8930633,187.0
max precision,0.9688271,1.0,0.0
max recall,0.0381216,1.0,390.0
max specificity,0.9688271,1.0,0.0
max absolute_mcc,0.3690763,0.7008582,215.0
max min_per_class_accuracy,0.2061066,0.8713956,281.0
max mean_per_class_accuracy,0.1689209,0.8745979,298.0


Gains/Lift Table: Avg response rate: 23.93 %, avg score: 24.02 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3705729,0.7282957,214.0
max f2,0.0923304,0.8059394,341.0
max f0point5,0.6112642,0.7637659,138.0
max accuracy,0.5157993,0.8742066,168.0
max precision,0.9698237,1.0,0.0
max recall,0.0339893,1.0,399.0
max specificity,0.9698237,1.0,0.0
max absolute_mcc,0.3738246,0.6444638,213.0
max min_per_class_accuracy,0.1825115,0.8425601,290.0
max mean_per_class_accuracy,0.1591801,0.8443311,302.0


Gains/Lift Table: Avg response rate: 23.93 %, avg score: 23.93 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100119,0.9677476,4.1791734,4.1791734,1.0,0.9689263,1.0,0.9689263,0.0418414,0.0418414,317.9173441,317.9173441
,2,0.0200033,0.9661127,4.1791734,4.1791734,1.0,0.9669258,1.0,0.9679271,0.0417558,0.0835972,317.9173441,317.9173441
,3,0.0300152,0.9643704,4.1620807,4.1734720,0.9959100,0.9652805,0.9986357,0.9670443,0.0416702,0.1252674,316.2080707,317.3471976
,4,0.0400066,0.9614407,4.1620457,4.1706183,0.9959016,0.9630237,0.9979529,0.9660402,0.0415847,0.1668521,316.2045681,317.0618326
,5,0.0500184,0.9544960,4.1278952,4.1620667,0.9877301,0.9586543,0.9959067,0.9645618,0.0413280,0.2081800,312.7895239,316.2066713
,6,0.1000164,0.8574442,3.6144203,3.8882995,0.8648649,0.9064410,0.9303992,0.9355073,0.1807136,0.3888936,261.4420273,288.8299547
,7,0.1500143,0.7077394,2.8956435,3.5574594,0.6928747,0.7893737,0.8512352,0.8868028,0.1447762,0.5336699,189.5643514,255.7459362
,8,0.2000123,0.4923502,2.4147476,3.2718107,0.5778051,0.6042167,0.7828846,0.8161635,0.1207324,0.6544023,141.4747635,227.1810674
,9,0.3000082,0.2153645,1.6121136,2.7186161,0.3857494,0.3338984,0.6505153,0.6554194,0.1612048,0.8156071,61.2113588,171.8616067
,10,0.4000041,0.1013642,0.9831839,2.2847802,0.2352580,0.1488377,0.5467062,0.5287805,0.0983144,0.9139215,-1.6816076,128.4780238



<bound method ModelBase.coef_norm of >


In [64]:
bm=stackedensemble_df(aml_leaderboard_df)

In [65]:
bm

['GBM_2_AutoML_20190414_131354',
 'GLM_grid_1_AutoML_20190414_131354_model_1',
 'DRF_1_AutoML_20190414_131354',
 'XRT_1_AutoML_20190414_131354',
 'DeepLearning_1_AutoML_20190414_131354']

In [66]:

aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190414_131354,0.928743,0.277279,0.167143,0.296535,0.087933
1,GBM_1_AutoML_20190414_131354,0.928644,0.277369,0.160988,0.296409,0.087858
2,GBM_3_AutoML_20190414_131354,0.928374,0.277844,0.165077,0.296980,0.088197
3,StackedEnsemble_AllModels_AutoML_20190414_131354,0.927854,0.292921,0.177949,0.300626,0.090376
4,GBM_grid_1_AutoML_20190414_131354_model_6,0.927796,0.279340,0.172041,0.297485,0.088498
5,GBM_grid_1_AutoML_20190414_131354_model_4,0.927535,0.279859,0.173029,0.297787,0.088677
6,StackedEnsemble_BestOfFamily_AutoML_20190414_1...,0.927298,0.294950,0.180466,0.301158,0.090696
7,GBM_4_AutoML_20190414_131354,0.927245,0.279931,0.169880,0.297962,0.088782
8,GBM_5_AutoML_20190414_131354,0.926568,0.280946,0.173531,0.298737,0.089244
9,GBM_grid_1_AutoML_20190414_131354_model_7,0.926406,0.281980,0.169133,0.298683,0.089212


In [67]:
#  Get best_models and coef_norm()
best_models={}
best_models=stackedensemble(mod_best)
bm=[]
if best_models is not None: 
  if 'Intercept' in best_models.keys():
    del best_models['Intercept']
  bm=list(best_models.keys())
else:
  best_models={}
  bm=stackedensemble_df(aml_leaderboard_df)   
  for b in bm:   
    best_models[b]=None

if mod_best.model_id not in bm:
    bm.append(mod_best.model_id)

In [68]:

bm

['GBM_2_AutoML_20190414_131354',
 'DRF_1_AutoML_20190414_131354',
 'XRT_1_AutoML_20190414_131354',
 'DeepLearning_1_AutoML_20190414_131354',
 'GLM_grid_1_AutoML_20190414_131354_model_1',
 'StackedEnsemble_BestOfFamily_AutoML_20190414_131354']

In [69]:
# Best of Family leaderboard

aml_leaderboard_df=aml_leaderboard_df.loc[aml_leaderboard_df['model_id'].isin(bm)]

In [70]:
aml_leaderboard_df

,model_id,auc,logloss,mean_per_class_error,rmse,mse
0,GBM_2_AutoML_20190414_131354,0.928743,0.277279,0.167143,0.296535,0.087933
6,StackedEnsemble_BestOfFamily_AutoML_20190414_1...,0.927298,0.294950,0.180466,0.301158,0.090696
13,DRF_1_AutoML_20190414_131354,0.916442,0.307283,0.187366,0.307515,0.094566
14,XRT_1_AutoML_20190414_131354,0.915869,0.301241,0.180619,0.308476,0.095158
16,DeepLearning_1_AutoML_20190414_131354,0.911404,0.309020,0.186417,0.314438,0.098871
20,GLM_grid_1_AutoML_20190414_131354_model_1,0.906792,0.317975,0.197513,0.318892,0.101692


In [71]:
# save leaderboard
leaderboard_stats=run_id+'_leaderboard.csv'
aml_leaderboard_df.to_csv(leaderboard_stats)

In [93]:
mod_best.params

{'model_id': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'ModelKeyV3',
    'schema_type': 'Key<Model>'},
   'name': 'GBM_2_AutoML_20190414_131354',
   'type': 'Key<Model>',
   'URL': '/3/Models/GBM_2_AutoML_20190414_131354'}},
 'training_frame': {'default': None,
  'actual': {'__meta': {'schema_version': 3,
    'schema_name': 'FrameKeyV3',
    'schema_type': 'Key<Frame>'},
   'name': 'automl_training_adult.hex',
   'type': 'Key<Frame>',
   'URL': '/3/Frames/automl_training_adult.hex'}},
 'validation_frame': {'default': None, 'actual': None},
 'nfolds': {'default': 0, 'actual': 5},
 'keep_cross_validation_models': {'default': True, 'actual': False},
 'keep_cross_validation_predictions': {'default': False, 'actual': True},
 'keep_cross_validation_fold_assignment': {'default': False, 'actual': False},
 'score_each_iteration': {'default': False, 'actual': False},
 'score_tree_interval': {'default': 0, 'actual': 5},
 'fold_assignment': {'default': 'AUTO

In [72]:
top=aml_leaderboard_df.iloc[0]['model_id']
print(top)

GBM_2_AutoML_20190414_131354


In [73]:
mod_best=h2o.get_model(top)
print(mod_best._id)
print(mod_best.algo)

GBM_2_AutoML_20190414_131354
gbm


In [74]:
meta_data['mod_best']=mod_best._id
meta_data['mod_best_algo']=mod_best.algo

In [75]:
meta_data['models']=bm

In [76]:
models_path=os.path.join(run_dir,'models')
for mod in bm:
  try:   
    m=h2o.get_model(mod) 
    h2o.save_model(m, path = models_path)
  except:    
    pass

In [77]:
print(models_path)

C:\Users\pkash\7\models


In [78]:
# GBM
 
mod,mod_id=get_model_by_algo("GBM",best_models)
if mod is not None:
    try:     
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_gbm_scoring_history.csv') 
    except:
        pass   
    try:     
        stats_gbm={}
        stats_gbm=gbm_stats(mod)
        n=run_id+'_gbm_stats.json'
        dict_to_json(stats_gbm,n)
        print(stats_gbm)
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_2_AutoML_20190414_131354', 'varimp': [('relationship', 6014.2978515625, 1.0, 0.25379080326533304), ('capital-gain', 4245.61572265625, 0.7059204295233316, 0.17915611285013525), ('marital-status', 3435.22900390625, 0.5711770664989241, 0.1449594865134985), ('education', 2700.3271484375, 0.44898460553229197, 0.11394816369180907), ('occupation', 1939.03173828125, 0.3224036763954905, 0.08182308800810803), ('capital-loss', 1331.7652587890625, 0.22143320661165344, 0.05619771137558999), ('age', 1086.8487548828125, 0.1807108297106456, 0.04586274663100956), ('educational-num', 1000.5480346679688, 0.16636157027175313, 0.04222103655175038), ('hours-per-week', 695.0702514648438, 0.11556964231232182, 0.029330512355531378), ('native-country', 551.0883178710938, 0.0916297016663254, 0.023254775588859553), ('workclass', 381.44647216796875, 0.06342327593051779, 0.016096244144124975), ('fnlwgt', 200.04261779785156, 0.03326117574071942, 0.008441380508786591), ('gender', 70.4

In [79]:
if mod is not None:
    try:    
        sh_df=mod.scoring_history()
        sh_df.to_csv(run_id+'_dl_scoring_history.csv') 
    except:
        pass 
    try:
        stats_dl={}
        stats_dl=dl_stats(mod)
        n=run_id+'_dl_stats.json'
        dict_to_json(stats_dl,n)
        print(stats_dl)
    except:
        pass    
    try:
        cf=mod.confusion_matrix()    
        cf_df.to_csv(run_id+'_dl_confusion_matrix.csv')
    except:
        pass

{'algo': 'gbm', 'model_id': 'GBM_2_AutoML_20190414_131354', 'varimp': [('relationship', 6014.2978515625, 1.0, 0.25379080326533304), ('capital-gain', 4245.61572265625, 0.7059204295233316, 0.17915611285013525), ('marital-status', 3435.22900390625, 0.5711770664989241, 0.1449594865134985), ('education', 2700.3271484375, 0.44898460553229197, 0.11394816369180907), ('occupation', 1939.03173828125, 0.3224036763954905, 0.08182308800810803), ('capital-loss', 1331.7652587890625, 0.22143320661165344, 0.05619771137558999), ('age', 1086.8487548828125, 0.1807108297106456, 0.04586274663100956), ('educational-num', 1000.5480346679688, 0.16636157027175313, 0.04222103655175038), ('hours-per-week', 695.0702514648438, 0.11556964231232182, 0.029330512355531378), ('native-country', 551.0883178710938, 0.0916297016663254, 0.023254775588859553), ('workclass', 381.44647216796875, 0.06342327593051779, 0.016096244144124975), ('fnlwgt', 200.04261779785156, 0.03326117574071942, 0.008441380508786591), ('gender', 70.4

In [80]:
# DRF

mod,mod_id=get_model_by_algo("DRF",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_drf_scoring_history.csv') 
    except:
         pass  
    try: 
         stats_drf={}
         stats_drf=drf_stats(mod)
         n=run_id+'_drf_stats.json'
         dict_to_json(stats_drf,n)
         print(stats_drf)
    except:
         pass

{'algo': 'drf', 'model_id': 'DRF_1_AutoML_20190414_131354', 'varimp': [('relationship', 34803.1875, 1.0, 0.15619088549234525), ('marital-status', 32336.115234375, 0.9291136116304002, 0.14511907772354318), ('capital-gain', 28277.732421875, 0.8125040966973212, 0.1269057343293127), ('occupation', 23072.083984375, 0.6629301981140376, 0.10354365466304742), ('education', 20855.880859375, 0.5992520328597776, 0.09359770564545664), ('age', 18098.119140625, 0.5200132643202581, 0.0812213322219461), ('educational-num', 14871.8125, 0.4273117943579162, 0.06674220754208587), ('hours-per-week', 12540.8720703125, 0.360336882083358, 0.05628133668815049), ('fnlwgt', 11413.7919921875, 0.3279524897593791, 0.05122318977493671), ('capital-loss', 8259.56640625, 0.23732212476946257, 0.037067552814667205), ('workclass', 7192.84228515625, 0.20667194018238272, 0.03228027334350737), ('native-country', 5951.33544921875, 0.1709997237815861, 0.02670859827639238), ('gender', 2667.28515625, 0.07663910543394911, 0.01197

In [81]:
# XRT

mod,mod_id=get_model_by_algo("XRT",best_models)
if mod is not None:
    try:     
         sh_df=mod.scoring_history()
         sh_df.to_csv(run_id+'_xrt_scoring_history.csv')
    except:
         pass     
    try:        
         stats_xrt={}
         stats_xrt=xrt_stats(mod)
         n=run_id+'_xrt_stats.json'
         dict_to_json(stats_xrt,n)
         print(stats_xrt)
    except:
         pass

{'algo': 'drf', 'model_id': 'XRT_1_AutoML_20190414_131354', 'varimp': [('marital-status', 48291.109375, 1.0, 0.19022389138329981), ('relationship', 38569.04296875, 0.7986779236990763, 0.15192762260797252), ('capital-gain', 32731.939453125, 0.6778046699848672, 0.12893464192229476), ('educational-num', 28025.39453125, 0.5803427358361448, 0.11039505354678186), ('occupation', 23177.50390625, 0.4799538508479336, 0.09129868919269381), ('education', 20829.85546875, 0.4313393446192703, 0.0820510486401998), ('age', 17333.79296875, 0.3589437723235158, 0.06827968115918037), ('hours-per-week', 10898.5244140625, 0.22568386924870681, 0.04293046383092884), ('capital-loss', 9095.80078125, 0.1883535271599877, 0.035829340892142916), ('native-country', 6563.33740234375, 0.1359119201709942, 0.025853694340302918), ('fnlwgt', 6309.8935546875, 0.13066366949014618, 0.024855351672836954), ('workclass', 6106.2646484375, 0.1264469739350961, 0.024053235435576664), ('gender', 4069.376708984375, 0.08426761699309947

In [82]:
# GLM

mod,mod_id=get_model_by_algo("GLM",best_models)
if mod is not None:
    try:     
         stats_glm={}
         stats_glm=glm_stats(mod)
         n=run_id+'_glm_stats.json'
         dict_to_json(stats_glm,n)
         print(stats_glm)
    except:
         pass

{'algo': 'glm', 'model_id': 'GLM_grid_1_AutoML_20190414_131354_model_1', 'coef': {'Intercept': -8.130286272637353, 'native-country.?': 0.065729266621754, 'native-country.Cambodia': 0.5572126666010029, 'native-country.Canada': 0.5901026055722867, 'native-country.China': -0.38607416793358634, 'native-country.Columbia': -1.0542652388600449, 'native-country.Cuba': 0.23552384656444758, 'native-country.Dominican-Republic': -0.525465533873249, 'native-country.Ecuador': -0.13221664295236543, 'native-country.El-Salvador': -0.29393625283317865, 'native-country.England': 0.49511135261503186, 'native-country.France': 0.5537852277774868, 'native-country.Germany': 0.21203688770410686, 'native-country.Greece': -0.004900891142730397, 'native-country.Guatemala': -0.18203537829312894, 'native-country.Haiti': 0.21250593926755865, 'native-country.Holand-Netherlands': -0.005788764323843744, 'native-country.Honduras': 0.021601287801425277, 'native-country.Hong': -0.1714622937455509, 'native-country.Hungary'

In [83]:
# split into training and test for showing how to predict
train, test = df.split_frame([0.8])

In [84]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions
predictions_df=predictions_test(mod_best,test,run_id)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [85]:

predictions_df.describe()

Rows:9622
Cols:3




,predict,<=50K,>50K
type,enum,real,real
mins,,0.0034870777420201726,0.0011571492900597383
mean,,0.7630776369812566,0.23692236301874345
maxs,,0.9988428507099403,0.9965129222579798
sigma,,0.3075185101111735,0.3075185101111734
zeros,,0,0
missing,0,0,0
0,<=50K,0.8548146934230575,0.14518530657694248
1,<=50K,0.822441824831469,0.17755817516853103
2,<=50K,0.9969266178965515,0.003073382103448532


In [86]:
# Update and save meta data

meta_data['end_time'] = time.time()
meta_data['execution_time'] = meta_data['end_time'] - meta_data['start_time']
  
n=run_id+'_meta_data.json'
dict_to_json(meta_data,n)

In [87]:
meta_data

{'start_time': 1555261986.631968,
 'target': None,
 'server_path': 'C:\\Users\\pkash',
 'data_path': 'C:/Users/pkash/Downloads/adult.csv',
 'test_path': None,
 'max_models': 9,
 'run_time': 500,
 'run_id': '7',
 'scale': False,
 'classification': False,
 'model_path': None,
 'balance': False,
 'balance_threshold': 0.2,
 'project': None,
 'end_time': 1555262925.0336282,
 'execution_time': 938.4016602039337,
 'run_path': 'C:\\Users\\pkash\\7',
 'nthreads': 1,
 'min_mem_size': 6,
 'analysis': 0,
 'x': ['age',
  'workclass',
  'fnlwgt',
  'education',
  'educational-num',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'gender',
  'capital-gain',
  'capital-loss',
  'hours-per-week',
  'native-country'],
 'variables': {'age': 'int',
  'workclass': 'enum',
  'fnlwgt': 'int',
  'education': 'enum',
  'educational-num': 'int',
  'marital-status': 'enum',
  'occupation': 'enum',
  'relationship': 'enum',
  'race': 'enum',
  'gender': 'enum',
  'capital-gain': 'int',
  'capita

In [88]:
# Save logs
h2o.download_all_logs(dirname=logs_path, filename=logfile)

Writing H2O logs to C:\Users\pkash\7\logs\7_autoh2o_log.zip


'C:\\Users\\pkash\\7\\logs\\7_autoh2o_log.zip'

In [89]:
# Clean up
os.chdir(server_path)

In [90]:
def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    print(stats_test)   
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
      print(cf)   
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      print(cf_df)    
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

In [91]:
h2o.cluster().shutdown()

H2O session _sid_86a6 closed.
